# Training

## Installation

In [ ]:
!pip install -q ranx pandas pytrec_eval tqdm sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.4 MB/s eta 0:00:00


## **Imports & logging**

In [ ]:
import os, gzip, logging
from collections import defaultdict
from google.colab import drive         # ↳ remove if not on Colab
from sentence_transformers import util # only used to download data
import ranx
import pandas as pd

Set up a simple logger:

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setFormatter(logging.Formatter("%(asctime)s  %(message)s",
                                           "%Y-%m-%d %H:%M:%S"))
    logger.addHandler(handler)

Mount Google Drive


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Define model-folder paths & locate .run files

In [ ]:
# root folder that contains your finetuned models
base_drive_path = "./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/"

model_dirs = {
    "MiniLM":       os.path.join(base_drive_path, "finetuned_models/cross-encoder-cross-encoder-ms-marco-MiniLM-L-2-v2-2025-05-18_19-56-28"),
    "TinyBERT":     os.path.join(base_drive_path, "finetuned_models/cross-encoder-cross-encoder-ms-marco-TinyBERT-L-2-v2-2025-05-19_21-08-20"),
    "DistilRoBERTa":os.path.join(base_drive_path, "finetuned_models/cross-encoder-distilroberta-base-2025-05-19_13-33-38"),
}

def find_run(model_dir):
    """Return full path to the first *.run found in `model_dir`
       (if several exist, pick the newest by mtime)."""
    if not os.path.exists(model_dir):
        logger.warning(f"Model dir not found → {model_dir}")
        return None
    runs = [f for f in os.listdir(model_dir) if f.endswith(".run")]
    if not runs:
        logger.warning(f"No .run file in {model_dir}")
        return None
    runs.sort(key=lambda f: os.path.getmtime(os.path.join(model_dir, f)),
              reverse=True)
    return os.path.join(model_dir, runs[0])

run_paths = {name: find_run(path) for name, path in model_dirs.items()}
print("Discovered run files:")
for k, v in run_paths.items():
    print(f"  {k:13s} → {v}")

Discovered run files:
  MiniLM        → ./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-cross-encoder-ms-marco-MiniLM-L-2-v2-2025-05-18_19-56-28/ranking.run
  TinyBERT      → ./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-cross-encoder-ms-marco-TinyBERT-L-2-v2-2025-05-19_21-08-20/ranking.run
  DistilRoBERTa → ./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/cross-encoder-distilroberta-base-2025-05-19_13-33-38/ranking.run


Download / load TREC-DL 2019 qrels & queries

In [ ]:
data_dir = "/content/trec2019-data"
os.makedirs(data_dir, exist_ok=True)

# Download qrels (ground-truth)
qrels_path = os.path.join(data_dir, "2019qrels-pass.txt")
if not os.path.exists(qrels_path):
    logger.info("Downloading qrels …")
    util.http_get("https://trec.nist.gov/data/deep/2019qrels-pass.txt", qrels_path)

# Load qrels into ranx
qrels = ranx.Qrels.from_file(qrels_path, kind="trec")
logger.info(f"Loaded qrels for {len(qrels)} queries.")

2025-05-21 16:09:56  Downloading qrels …
INFO:__main__:Downloading qrels …


0.00B [00:00, ?B/s]

2025-05-21 16:10:16  Loaded qrels for 43 queries.
INFO:__main__:Loaded qrels for 43 queries.


Load the three runs into ranx


In [ ]:
runs_to_fuse, run_names = [], []
for name, path in run_paths.items():
    if path and os.path.exists(path):
        runs_to_fuse.append(ranx.Run.from_file(path, kind="trec"))
        run_names.append(name)
    else:
        logger.warning(f"Skipping {name} – run file missing.")

assert len(runs_to_fuse) >= 2, "Need at least two runs to build ensembles."

Evaluate each individual model

In [ ]:
metrics = ["ndcg@10", "recall@100", "map@1000"]
results_summary = []

print("\n Individual-model scores")
for run, name in zip(runs_to_fuse, run_names):
    run.name = name
    scores = ranx.evaluate(qrels, run, metrics)
    for m, v in scores.items():
        pct = v * 100
        print(f"{name:13s}  {m:10s}  {pct:6.2f}")
    results_summary.append({"Method": name, **scores})


 Individual-model scores
MiniLM         ndcg@10      67.92
MiniLM         recall@100   49.74
MiniLM         map@1000     43.97
TinyBERT       ndcg@10      67.05
TinyBERT       recall@100   49.37
TinyBERT       map@1000     43.97
DistilRoBERTa  ndcg@10      63.17
DistilRoBERTa  recall@100   49.75
DistilRoBERTa  map@1000     41.89


Fuse with your five Comb rules

In [ ]:
fusion_rules = {               # prettier names → ranx method keyword
    "CombANZ": "anz",
    "CombSUM": "sum",
    "CombMNZ": "mnz",
    "CombMIN": "min",
    "CombMAX": "max",
}

print("\n Ensemble scores (min-max normalised)")
for pretty, method_key in fusion_rules.items():
    fused = ranx.fuse(
        runs=runs_to_fuse,
        norm="min-max",
        method=method_key,
    )
    fused.name = pretty
    scores = ranx.evaluate(qrels, fused, metrics)
    for m, v in scores.items():
        pct = v * 100
        print(f"{pretty:13s}  {m:10s}  {pct:6.2f}")
    results_summary.append({"Method": pretty, **scores})


 Ensemble scores (min-max normalised)
CombANZ        ndcg@10      69.54
CombANZ        recall@100   51.18
CombANZ        map@1000     45.55
CombSUM        ndcg@10      69.54
CombSUM        recall@100   51.18
CombSUM        map@1000     45.55
CombMNZ        ndcg@10      69.54
CombMNZ        recall@100   51.18
CombMNZ        map@1000     45.55
CombMIN        ndcg@10      68.88
CombMIN        recall@100   50.20
CombMIN        map@1000     43.76
CombMAX        ndcg@10      64.83
CombMAX        recall@100   50.38
CombMAX        map@1000     43.15


 Summary table

In [ ]:
print("\n=== Summary Table (percent) ===")

# Build the DataFrame
df = pd.DataFrame(results_summary)

# Scale each metric column to 0–100 and keep two decimals
for col in ["ndcg@10", "recall@100", "map@1000"]:
    df[col] = (df[col] * 100).round(2)

print(df.to_string(index=False))


=== Summary Table (percent) ===
       Method  ndcg@10  recall@100  map@1000
       MiniLM    67.92       49.74     43.97
     TinyBERT    67.05       49.37     43.97
DistilRoBERTa    63.17       49.75     41.89
      CombANZ    69.54       51.18     45.55
      CombSUM    69.54       51.18     45.55
      CombMNZ    69.54       51.18     45.55
      CombMIN    68.88       50.20     43.76
      CombMAX    64.83       50.38     43.15
      CombANZ    69.54       51.18     45.55
      CombSUM    69.54       51.18     45.55
      CombMNZ    69.54       51.18     45.55
      CombMIN    68.88       50.20     43.76
      CombMAX    64.83       50.38     43.15


**TASK 3**


imports & setup

In [ ]:
import itertools
import pandas as pd
import ranx      # already installed earlier

# Metrics to report
pair_metrics = ["ndcg@10", "recall@100", "map@1000"]
pair_results = []

fuse & evaluate each pair

In [ ]:
print("\n🔹 CombSUM scores for each 2-model pair (percent)")

for i, j in itertools.combinations(range(len(runs_to_fuse)), 2):
    run_pair  = [runs_to_fuse[i], runs_to_fuse[j]]
    pair_name = f"{run_names[i]} + {run_names[j]}"

    # Combine the two runs with CombSUM (min-max normalised)
    fused = ranx.fuse(
        runs=run_pair,
        norm="min-max",
        method="sum",
    )

    # Evaluate on the three metrics
    scores = ranx.evaluate(qrels, fused, pair_metrics)
    scores_pct = {k: round(v * 100, 2) for k, v in scores.items()}

    # Print results line-by-line
    print(f"\n{pair_name}")
    for m in pair_metrics:
        print(f"  {m:11s}: {scores_pct[m]:6.2f}")

    # Save for the final table
    pair_results.append({"Pair": pair_name, **scores_pct})


🔹 CombSUM scores for each 2-model pair (percent)

MiniLM + TinyBERT
  ndcg@10    :  69.66
  recall@100 :  50.61
  map@1000   :  45.56

MiniLM + DistilRoBERTa
  ndcg@10    :  68.56
  recall@100 :  50.71
  map@1000   :  44.60

TinyBERT + DistilRoBERTa
  ndcg@10    :  68.08
  recall@100 :  50.34
  map@1000   :  44.43


display the summary table

In [ ]:
import pandas as pd
summary_df = pd.DataFrame(pair_results)
summary_df

,Pair,ndcg@10,recall@100,map@1000
0,MiniLM + TinyBERT,69.66,50.61,45.56
1,MiniLM + DistilRoBERTa,68.56,50.71,44.60
2,TinyBERT + DistilRoBERTa,68.08,50.34,44.43


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
base_path = "./gdrive/MyDrive/cross-encoder-reranker-ir-course-2025model/"

Mounted at /content/gdrive
